# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
import zipfile

from torch import nn, optim
import torch.nn.functional as F
import numpy as np

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
#with open(os.path.join('..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
#    twits = json.load(f)

fname = os.path.join('..', 'data', 'project_6_stocktwits', 'twits.zip')
with zipfile.ZipFile(fname) as myzip:
    for filename in myzip.namelist():
        if filename.endswith('.json'):
            twits = json.load(myzip.open(filename))

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

print(f'Number of twits: {len(twits["data"])}')

Number of twits: 1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 

    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'(http|https):\/\/[\w\$-_\.\+!\*\'\(\),/&?=:%]+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\w+', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@\w+', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized = []
    for t in tokens:
        lem_t = wnl.lemmatize(t)
        if len(lem_t) > 1:
            lemmatized.append(lem_t)

    assert type(lemmatized) == list, 'Tokens should be list'
    return lemmatized

[nltk_data] Downloading package wordnet to /home/pdippold/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


>Note: You must ensure that after preprocessing the text should NOT include:
- Numbers
- URLs
- Single character tokens
- Ticker symbols (these should be removed even if they don't appear at the beginning)

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# Preprocess all twits
tokenized = [preprocess(message) for message in messages]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""
bow = Counter()
for tokens in tokenized:
    bow.update(tokens)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the **number of messages** calculated in the code block above (i.e. `len(messages))`. Then remove words that only appear in some small fraction of the messages.

>Note: There is not an exact number for low and high-frequency cut-offs, however there is a correct optimal range.
You should ideally set up low-frequency cut-off from 0.0000002 to 0.000007 (inclusive) and high-frequency from 5 to 20 (inclusive). If the number is too big, we lose lots of important words that we can use in our data.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# Dictionary that contains the frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = Counter({token: count/len(messages) for token, count in bow.items()})

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 2e-7

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 5

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = freqs.most_common(high_cutoff)

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

[('the', 0.25763270263112), ('to', 0.24516831286619595), ('is', 0.18403627883540805), ('for', 0.17671009877197175), ('on', 0.1561204384984593)]


99274

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {w: ii for ii, w in enumerate(filtered_words, 1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii: w for w, ii in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in message if word in vocab] for message in tokenized]

assert set(vocab.keys()) == set(id2vocab.values()), 'Check vocab and id2vocab dictionaries'

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [10]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [11]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19465436984436835

Finally let's convert our tokens into integer ids which we can pass to the network.

In [12]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [13]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, self.lstm_size, self.lstm_layers,
                            batch_first=False, dropout=self.dropout)
        self.dropout = nn.Dropout(self.dropout)
        self.fc = nn.Linear(self.lstm_size, self.output_size)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # Create two new tensors with sizes lstm_layers x batch_size x lstm_size,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new_zeros(self.lstm_layers, batch_size, self.lstm_size),
                  weight.new_zeros(self.lstm_layers, batch_size, self.lstm_size))
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        x = nn_input.permute(1,0) # reshape to get the batch to dim 0
        x = self.embedding(nn_input)
        x, hidden = self.lstm(x, hidden_state)
        x = x[-1, :, :] # getting the last time step output
        x = self.dropout(x)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        
        return output, hidden

### View Model

In [14]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.5690, -2.0820, -1.4428, -1.3775, -1.7227],
        [-1.7176, -2.1311, -1.3430, -1.3704, -1.6781],
        [-1.6024, -2.1508, -1.3952, -1.3149, -1.7963],
        [-1.5804, -2.0842, -1.4252, -1.3454, -1.7792]],
       grad_fn=<LogSoftmaxBackward0>)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [15]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    # return only full batches
    num_full_batches = len(messages) // batch_size
    max_idx = num_full_batches * batch_size
    messages = messages[:max_idx]
    labels = labels[:max_idx]
    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [16]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   
split_idx = int(len(token_ids)*0.9)
train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

In [17]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
text_batch = text_batch
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'User {device} device.')

model = TextClassifier(len(vocab)+1, 784, 1024, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-0.5, 0.5)
model.to(device)

User cuda device.


TextClassifier(
  (embedding): Embedding(99275, 784)
  (lstm): LSTM(784, 1024, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=1024, out_features=5, bias=True)
)

In [19]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 512
learning_rate = 0.001
clip=4 # gradient clipping

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, eps=1e-6)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=30, shuffle=True):
        steps += 1
        hidden = model.init_hidden(batch_size)
        
        # set device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # zero accumulated gradients
        optimizer.zero_grad()

        # get the output from the model
        output, _ = model(text_batch, hidden)

        # calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            # get validation loss and accuracy
            valid_losses = []
            eq_list = []
            model.eval()
            for text_batch, labels in dataloader(
                valid_features, valid_labels, batch_size=batch_size, sequence_length=30, shuffle=True):
                valid_hidden = model.init_hidden(batch_size)
                # set device
                text_batch, labels = text_batch.to(device), labels.to(device)
                for each in hidden:
                    each.to(device)

                output, _ = model(text_batch, valid_hidden)
                valid_loss = criterion(output, labels)
                valid_losses.append(valid_loss.item())
                # collect data to calculate validation accuracy
                top_p, top_class = torch.exp(output).topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape).to(device)
                eq_list.append(equals.view(-1))
                
            # calculate validation accuracy    
            accuracy = torch.mean(torch.cat(eq_list, dim=0).type(torch.FloatTensor)) 
            model.train()
            print(f'Epoch: {epoch+1}/{epochs}... ',
                  f'Step: {steps}... ',
                  f'Loss: {loss.item():.6f}... ',
                  f'Valid Loss: {np.mean(valid_losses):.6f}... ',
                  f'Valid Accuracy: {accuracy*100:.3f}')

Starting epoch 1
Epoch: 1/3...  Step: 100...  Loss: 1.062703...  Valid Loss: 1.059996...  Valid Accuracy: 57.938
Epoch: 1/3...  Step: 200...  Loss: 0.877123...  Valid Loss: 0.935597...  Valid Accuracy: 63.619
Epoch: 1/3...  Step: 300...  Loss: 0.870843...  Valid Loss: 0.884587...  Valid Accuracy: 65.014
Epoch: 1/3...  Step: 400...  Loss: 0.791059...  Valid Loss: 0.846038...  Valid Accuracy: 67.251
Epoch: 1/3...  Step: 500...  Loss: 0.785155...  Valid Loss: 0.799552...  Valid Accuracy: 68.835
Epoch: 1/3...  Step: 600...  Loss: 0.734225...  Valid Loss: 0.776929...  Valid Accuracy: 70.601
Epoch: 1/3...  Step: 700...  Loss: 0.730264...  Valid Loss: 0.771725...  Valid Accuracy: 70.579
Epoch: 1/3...  Step: 800...  Loss: 0.688225...  Valid Loss: 0.750345...  Valid Accuracy: 71.433
Epoch: 1/3...  Step: 900...  Loss: 0.622013...  Valid Loss: 0.738477...  Valid Accuracy: 72.032
Epoch: 1/3...  Step: 1000...  Loss: 0.658201...  Valid Loss: 0.733197...  Valid Accuracy: 71.922
Epoch: 1/3...  Step: 1

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [20]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """
    
    # Process several preprocessing steps to the text
    tokens = preprocess(text)
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    # Check if there are any tokens left to predict the sentiment class
    if (len(tokens) < 1):
        return torch.ones(1, 5)*(-1)

    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1)
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [21]:
torch.set_printoptions(precision=6, sci_mode=False)
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[    0.000038,     0.007864,     0.004313,     0.845940,     0.141844]],
       grad_fn=<ExpBackward0>)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?

**Answer**: The model returns the log probability of each sentiment class (five values). By exposing the log probabilites the predict function returns class probabilities which can be taken to select the most probable sentiment class. In the example text above the model predicts class 4, i.e. the second best sentiment, which corresponds quite good to the meaning of the twit.

The probability of 84.6% for sentiment 4 is quite hight, i.e. the model is pretty sure that the twit belongs to this class. But the result may be less clear for other twits. Thus, I might be reasonable to specify a threshold, e.g. 65%, and if the probabilities of all classes are below this value, the model might predict "not sure".

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [22]:
with open(os.path.join('..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [23]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [24]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [25]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

max_twits = 20
for ii, twit_score in enumerate(score_stream, 1):
    _, top_class = twit_score["score"].topk(1, dim=1)
    if top_class.sum() == 0:
        print(f'Not enough information in twit for {twit_score["symbol"]} to determine sentiment!')
    else:
        print(f'{twit_score["symbol"]}: Sentiment score for twit on', 
              f'{twit_score["timestamp"]} is {top_class.item()}')
    if ii >= max_twits:
        break

$AAPL: Sentiment score for twit on 2018-11-01T00:00:18Z is 4
$AMZN: Sentiment score for twit on 2018-11-01T00:00:42Z is 3
$AAPL: Sentiment score for twit on 2018-11-01T00:00:55Z is 2
$MU: Sentiment score for twit on 2018-11-01T00:02:10Z is 4
$AMZN: Sentiment score for twit on 2018-11-01T00:03:35Z is 2
$MU: Sentiment score for twit on 2018-11-01T00:04:21Z is 2
$AAPL: Sentiment score for twit on 2018-11-01T00:05:40Z is 2
$AAPL: Sentiment score for twit on 2018-11-01T00:07:32Z is 3
$AMZN: Sentiment score for twit on 2018-11-01T00:07:32Z is 3
$GOOG: Sentiment score for twit on 2018-11-01T00:07:32Z is 3
$FB: Sentiment score for twit on 2018-11-01T00:08:52Z is 4
$MU: Sentiment score for twit on 2018-11-01T00:08:56Z is 2
$FB: Sentiment score for twit on 2018-11-01T00:09:31Z is 3
Not enough information in twit for $AAPL to determine sentiment!
$AAPL: Sentiment score for twit on 2018-11-01T00:10:17Z is 2
$AAPL: Sentiment score for twit on 2018-11-01T00:11:02Z is 4
$MU: Sentiment score for twit 

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.